In [1]:
import numpy as np

#visuals
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from mpl_toolkits.mplot3d import Axes3D
import matplotlib
matplotlib.rcParams.update({'font.size': 10})

#Interactive
from ipywidgets.widgets import * 
import ipywidgets as widgets

#is vectorization useful here?
import time

#Hugo code
import Utility
import Datasets 

ModuleNotFoundError: No module named 'Datasets'

In [ ]:
#load data

#split into timeslices

# convert each LM timeslice to histo image (list mode histo image bins)

#new!!! normalize sinogram

# calculate center of mass (for valid image points in FOV)

#calculate center of mass for reconstruction

# plot time curves for each dimension and compare with recon COM

#plot the points in 3D

In [ ]:
#load data
#define dataset (phantom)
LM_data, LM_paths = Datasets.KEX_listmode_files()

In [ ]:
#split into timeslices
#can be made more efficient
print(Utility.timeSlices.__doc__)
#each time tag is 1 ms
timeTagsPerSlice = 1000
time_slices = []
first_events = []
for L in LM_data:
    #print(L)
    ts = Utility.timeSlices(L, timeTagsPerSlice=timeTagsPerSlice)
    time_slices.append(ts)

In [ ]:
#calculate COM of recon for comparison
recon_COM = []
recon, recon_path = Datasets.KEX_reconstructions()
for r in recon:
    com =Utility.CenterOfMassMultiplication(r, verbose=False, convert_to_datatype=np.int64)
    print("center of mass", com)
    recon_COM.append(com)

In [ ]:
#calculate center of mass for each time slice of histo image
#this takes some time
verbose = False
if verbose:
    print("documentation\n", Utility.events.__doc__)
    print(Utility.binAddresToProjectionSpace.__doc__)
    print(Utility.imageBinFromHistogramBin.__doc__)
    print(Utility.bValidBin.__doc__)
time_a = time.time()
COM_traces = []
total_event_count = 0
if True:
    for ts in time_slices:
        COM = np.zeros((3,len(ts)))
        #this step can maybe be vectorized? No, unfortunately, the number of events per time slice differs.
        #but it might be possible at a prior step. Do not split the data into timeslices before necessary
        #so I need to find the index of the first event per time slice. 
        #I fail to see the benefits of this, as a for loop would still be necessary  

        for k,mini_gate in enumerate(ts):
            events = Utility.events(mini_gate)
            total_event_count += len(events)
            histogram_bins = Utility.binAddresToProjectionSpace(np.array(events,dtype=int))
            histo_image_bins = Utility.imageBinFromHistogramBin(histogram_bins, delays=True)
            valid_image_bins = histo_image_bins[:,Utility.bValidBin(histo_image_bins, Utility.IMAGE_SHAPE)]
            COM[:,k] = np.mean(valid_image_bins, axis=1)
        COM_traces.append(COM)
        
        # elapsed time 26.202388763427734 for 74789598events
        # elapsed time 12.284845113754272 for 33629375events
else:
    for k,L in enumerate(LM_data):
        events = Utility.events(L)
        histogram_bins = Utility.binAddresToProjectionSpace(np.array(events,dtype=int))
        histo_image_bins = Utility.imageBinFromHistogramBin(histogram_bins, delays=True)
        bValidBin = Utility.bValidBin(histo_image_bins, Utility.IMAGE_SHAPE)
        event_count = 0
        valid_event_count = 0
        first_event = [event_count]
        first_valid_event = [valid_event_count]
        for j,ts in enumerate(time_slices[k]):
            number_of_events_in_slice = len(ts)
            valid_event_count += np.sum(bValidBin[event_count:event_count+number_of_events_in_slice])
            first_valid_event.append(valid_event_count)
            event_count += number_of_events_in_slice
            first_event.append(event_count)
        
        valid_image_bins = histo_image_bins[:,bValidBin]
        
        split_image_bins = np.split( valid_image_bins, first_valid_event[1:-1],axis=1)
        COM = np.zeros((3,len(split_image_bins)))
        for k,img_bins in enumerate(split_image_bins):
            if len(img_bins) > 0:
                COM[:,k] = np.mean(img_bins, axis=1)
        COM_traces.append(COM)
        total_event_count += event_count
        
        #slightly faster, but does not work correctly
        # elapsed time 24.627336025238037 for 75366047events
time_b = time.time()
print("elapsed time " + str(time_b-time_a) +" for " + str(total_event_count) + "events")

In [ ]:
print(COM_traces[0].shape)
print(recon_COM[1])


In [ ]:
#plot the COM
plt.figure(figsize=(15,10))
xlabel = "time slice number\n" + str(timeTagsPerSlice) + " ms per slice"
legends = ["histo image", "recon"]
reset_ylim = [False,True]
columns = len(reset_ylim)

for d,dim_str in enumerate(Utility.IMAGE_DIMENSION_TITLES):

    for k,redo_y in enumerate(reset_ylim):
        plot_legends = []
        hline_legends = []
        b = 0
        plt.subplot(3,columns,columns*d+1+k)
        if redo_y:
                plt.ylim(0,Utility.IMAGE_SHAPE[d])
        for gate_number, com in enumerate(COM_traces):
            a = b
            b += len(com[d])
            plt.plot(np.arange(a,b), com[d])
            plot_legends.append("histo. image gate " + str(gate_number))
            plt.hlines(recon_COM[gate_number][d], a, b)
            hline_legends.append("recon. gate " + str(gate_number))
        plt.legend(plot_legends+hline_legends)
        plt.ylabel(dim_str)
        plt.xlabel(xlabel)
    
plt.suptitle(phantom_str)
plt.show()

In [ ]:
print(recon_COM[0][1])
print(recon_COM[1][1])